In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%shell

train_dataset="ffhq"
test_dataset="frll"

# Both, with custom grid and parallelism
python drive/MyDrive/mad/tests/occlusion_heatmaps_builder.py \
  --heatmap-mode diff \
  --regular /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/${train_dataset}-cropped/${test_dataset}-cropped/test_using_train_model_with_balanced_data.txt \
  --grid /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/${test_dataset}-grid-occlusion/${train_dataset}-cropped/test_using_train_model_with_balanced_data.txt \
  --landmark /content/drive/MyDrive/mad/code/ViT-SVM/mad_scores/${test_dataset}-landmark-occlusion/${train_dataset}-cropped/test_using_train_model_with_balanced_data.txt \
  --output output_diff \
  --multiprocessing --workers 12

Streaming output truncated to the last 5000 lines.
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs-occluded-regions/right_eye/morph_amsl/123_130.png' (suffix='morph_amsl/123_130.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs-occluded-regions/right_eye/morph_amsl/123_131.png' (suffix='morph_amsl/123_131.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs-occluded-regions/right_eye/morph_amsl/123_132.png' (suffix='morph_amsl/123_132.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs-occluded-regions/right_eye/morph_amsl/123_138.png' (suffix='morph_amsl/123_138.png')
[landmark] No match found for '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs-occluded-regions/right_eye/morph_amsl/123_141.png' (suffix='morph_amsl/123_141.png')
[landmark] No match foun

In [ ]:
from pathlib import Path
from zipfile import ZipFile, ZIP_DEFLATED
from tqdm import tqdm

# --- Paths ---
src = Path("output_diff")  # folder to zip
zip_local = Path("/content/output_diff.zip")  # temporary zip in Colab local storage
zip_drive = Path("/content/drive/MyDrive/mad/tests/output_diff_heatmaps.zip")  # final zip on Drive

# --- 1. Zip with progress bar ---
files = [f for f in src.rglob("*") if f.is_file()]

print("Zipping files...")
with ZipFile(zip_local, "w", compression=ZIP_DEFLATED) as zipf:
    for f in tqdm(files, desc="Zipping", unit="file"):
        rel = f.relative_to(src)
        zipf.write(f, rel)

# --- 2. Copy zip to Drive with progress bar ---
print("Copying zip to Drive...")
buffer_size = 1024 * 1024  # 1 MB chunks
total_size = zip_local.stat().st_size

with open(zip_local, "rb") as fsrc, open(zip_drive, "wb") as fdst:
    with tqdm(total=total_size, unit="B", unit_scale=True, desc="Copying") as pbar:
        while True:
            buf = fsrc.read(buffer_size)
            if not buf:
                break
            fdst.write(buf)
            pbar.update(len(buf))

print(f"\n✅ Zip saved at: {zip_drive}")

Zipping files...


Zipping: 100%|██████████| 4364/4364 [02:15<00:00, 32.11file/s]


Copying zip to Drive...


Copying: 100%|██████████| 2.24G/2.24G [00:09<00:00, 231MB/s]


✅ Zip saved at: /content/drive/MyDrive/mad/tests/output_diff_heatmaps.zip


In [ ]:
import time
from google.colab import drive, runtime

drive.flush_and_unmount()

runtime.unassign()